In [2]:
from platform import python_version
print(python_version())

3.11.8


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import json
import re
import nltk
import tkinter as tk
from tkinter import ttk
import ast

In [3]:
movies_df = pd.read_csv('../../edit_data/Lee/cleaned_data/movies_main.csv')  

In [5]:
movies_df.columns

Index(['Id', 'Original_Title', 'Original_Language', 'Spoken_Languages',
       'Budget', 'Revenue', 'Runtime', 'Release_Date', 'Production_Companies',
       'Production_Countries', 'imdb_id', 'Popularity', 'Vote_Average',
       'Vote_Count', 'Belongs_To_Collection', 'Tagline', 'Overview', 'Genres',
       'Genres_Parse', 'Belongs_To_Collection_Parse', 'Spoken_Languages_Parse',
       'Production_Companies_Parse', 'Production_Countries_Parse',
       'Production_Countries_Code_Parse', 'Keywords', 'Keywords_parse',
       'Cast_parse', 'Director_parse'],
      dtype='object')

In [6]:
movies_df.head(2)

,Id,Original_Title,Original_Language,Spoken_Languages,Budget,Revenue,Runtime,Release_Date,Production_Companies,Production_Countries,...,Genres_Parse,Belongs_To_Collection_Parse,Spoken_Languages_Parse,Production_Companies_Parse,Production_Countries_Parse,Production_Countries_Code_Parse,Keywords,Keywords_parse,Cast_parse,Director_parse
0,862,Toy Story,en,"[{'iso_639_1': 'en', 'name': 'English'}]",30000000.0,373554033.0,81.0,1995-10-30,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"['Animation', 'Comedy', 'Family']",['Toy Story Collection'],['English'],['Pixar Animation Studios'],['United States of America'],['US'],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","['jealousy', 'toy', 'boy', 'friendship', 'frie...","['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...",['John Lasseter']
1,8844,Jumanji,en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",65000000.0,262797249.0,104.0,1995-12-15,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"['Adventure', 'Fantasy', 'Family']",NaN,"['English', 'Français']","['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],['US'],"[{'id': 10090, 'name': 'board game'}, {'id': 1...","['board game', 'disappearance', ""based on chil...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",['Joe Johnston']


In [6]:
# # Initialize the main window
# root = tk.Tk()
# root.title("Movie Recommender")

# # Define the function to show recommendations
# def show_recommendations():
#     genre = genre_input.get()
#     year = year_input.get()
#     rating = rating_input.get()
#     recommendations = recommend_movies(genre, year, rating)
#     result_var.set("\n".join(recommendations))

# # Simple recommendation function considering multiple inputs
# def recommend_movies(genre, year, rating):
#     filtered_movies = movies_df
    
#     if genre:
#         filtered_movies = filtered_movies[filtered_movies['genres'].str.contains(genre, case=False, na=False)]
#     if year:
#         filtered_movies = filtered_movies[filtered_movies['release_date'].str.contains(year, na=False)]
#     if rating:
#         filtered_movies = filtered_movies[filtered_movies['vote_average'] >= float(rating)]
    
#     top_movies = filtered_movies.head(10)['title'].tolist()
#     return top_movies

# # Add a label and entry for genre input
# ttk.Label(root, text="Enter your preferred genre:").grid(column=0, row=0, padx=10, pady=10)
# genre_input = ttk.Entry(root)
# genre_input.grid(column=1, row=0, padx=10, pady=10)

# # Add a label and entry for year input
# ttk.Label(root, text="Enter release year:").grid(column=0, row=1, padx=10, pady=10)
# year_input = ttk.Entry(root)
# year_input.grid(column=1, row=1, padx=10, pady=10)

# # Add a label and entry for rating input
# ttk.Label(root, text="Enter minimum rating:").grid(column=0, row=2, padx=10, pady=10)
# rating_input = ttk.Entry(root)
# rating_input.grid(column=1, row=2, padx=10, pady=10)

# # Add a button to trigger recommendation
# ttk.Button(root, text="Get Recommendations", command=show_recommendations).grid(column=0, row=3, columnspan=2, padx=10, pady=10)

# # Add a label to display the results
# result_var = tk.StringVar()
# ttk.Label(root, textvariable=result_var).grid(column=0, row=4, columnspan=2, padx=10, pady=10)

# # Run the application
# root.mainloop()


In [4]:
# Define a function to extract genres from a single row entry
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)  # Convert string to list of dicts
        return [genre['name'] for genre in genres]
    except:
        return []

# Use the function to extract genres for the entire column
all_genres = set()
movies_df['Genres'].dropna().apply(lambda x: all_genres.update(extract_genres(x)))

# Convert the set to a sorted list
all_genres = sorted(list(all_genres))

# Display all unique genres
print(all_genres)

['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']


In [5]:
# Search for movies produced by "Carousel Productions"
carousel_movies = movies_df[movies_df['Production_Companies'].str.contains("Carousel Productions", case=False, na=False)]

# Display the titles of the movies
carousel_movie_titles = carousel_movies['Original_Title'].tolist()
carousel_movie_titles


['The Deadly Companions', 'The Incredible Burt Wonderstone']

In [6]:
df_1 = pd.read_csv("../../source_data/movies_metadata.csv")

#add the TMDB prefix to fetch the poster image online
df_1['poster_url'] = 'https://image.tmdb.org/t/p/original' + df_1['poster_path'].astype(str)
poster_url = df_1['poster_url']

### merge df_1['poster_url'] to movies_df

# Exclude if it is identical in 3 columns (id, Vote_Count, Original_Title)
df_1 = df_1.loc[~df_1.duplicated(subset=['id'])].reset_index(drop=True).copy()


movies_df = pd.concat([movies_df, df_1], axis = 1)
movies_df.head()

/var/folders/nq/ln_gwthx2kb0qfqrxc4zszmr0000gn/T/ipykernel_25075/14460332.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv("../../source_data/movies_metadata.csv")


,Id,Original_Title,Original_Language,Spoken_Languages,Budget,Revenue,Runtime,Release_Date,Production_Companies,Production_Countries,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,poster_url
0,862.0,Toy Story,en,"[{'iso_639_1': 'en', 'name': 'English'}]",30000000.0,373554033.0,81.0,1995-10-30,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,https://image.tmdb.org/t/p/original/rhIRbceoE9...
1,8844.0,Jumanji,en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",65000000.0,262797249.0,104.0,1995-12-15,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,https://image.tmdb.org/t/p/original/vzmL6fP7aP...
2,15602.0,Grumpier Old Men,en,"[{'iso_639_1': 'en', 'name': 'English'}]",0.0,0.0,101.0,1995-12-22,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,https://image.tmdb.org/t/p/original/6ksm1sjKMF...
3,31357.0,Waiting to Exhale,en,"[{'iso_639_1': 'en', 'name': 'English'}]",16000000.0,81452156.0,127.0,1995-12-22,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,https://image.tmdb.org/t/p/original/16XOMpEaLW...
4,11862.0,Father of the Bride Part II,en,"[{'iso_639_1': 'en', 'name': 'English'}]",0.0,76578911.0,106.0,1995-02-10,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,https://image.tmdb.org/t/p/original/e64sOI48hQ...


In [15]:
movies_df['poster_url'].head()

0    https://image.tmdb.org/t/p/original/rhIRbceoE9...
1    https://image.tmdb.org/t/p/original/vzmL6fP7aP...
2    https://image.tmdb.org/t/p/original/6ksm1sjKMF...
3    https://image.tmdb.org/t/p/original/16XOMpEaLW...
4    https://image.tmdb.org/t/p/original/e64sOI48hQ...
Name: poster_url, dtype: object

In [14]:
## TEST TEST GUI
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import requests
from io import BytesIO
import pandas as pd
import ast

# Initialize the main window
root = tk.Tk()
root.title("Movie Recommender")
root.geometry("800x600")  # Increase window size to fit posters

# Extract unique genres from the dataset (assuming 'movies_df' is your DataFrame)
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [genre['name'] for genre in genres]
    except:
        return []

# Extract the release year from the release_date column
movies_df['Release_Year'] = pd.to_datetime(movies_df['Release_Date'], errors='coerce').dt.year

all_genres = set()
all_release_years = sorted(movies_df['Release_Year'].dropna().astype(int).unique(), reverse=True)

movies_df['Genres'].dropna().apply(lambda x: all_genres.update(extract_genres(x)))

all_genres = sorted(list(all_genres))

# Define the function to show recommendations
def show_recommendations():
    genre = genre_input.get()
    release_year = release_year_input.get()
    keywords = keyword_input.get()
    
    # Clear previous results
    for widget in result_frame.winfo_children():
        widget.destroy()
    
    recommendations = recommend_movies(genre, release_year, keywords)
    
    for movie in recommendations:
        movie_title = movie['title']
        match_percentage = movie['match_percentage']
        poster_url = movie['poster_url']
        
        # Download and display the poster image
        response = requests.get(poster_url)
        img_data = response.content
        img = Image.open(BytesIO(img_data))
        img = img.resize((100, 150), Image.ANTIALIAS)
        img_tk = ImageTk.PhotoImage(img)
        
        # Create a label for the image
        image_label = tk.Label(result_frame, image=img_tk)
        image_label.image = img_tk  # Keep a reference to avoid garbage collection
        image_label.pack(side="left", padx=10, pady=10)
        
        # Create a label for the movie title and match percentage
        text_label = tk.Label(result_frame, text=f"{movie_title} - {match_percentage:.2f}%", anchor='w', justify='left')
        text_label.pack(side="left", padx=10, pady=10)

# Calculate match percentage based on genre, release year, and keywords
def calculate_match_percentage(row, genre, release_year, keywords):
    match_score = 0
    total_criteria = 3  # We have three criteria now: genre, release year, and keywords
    
    # Check genre match
    if genre and genre != "Any":
        if genre in extract_genres(row['Genres']):
            match_score += 1
    
    # Check release year match
    if release_year and release_year != "Any":
        if pd.notna(row['Release_Year']) and row['Release_Year'] == int(release_year):
            match_score += 1
            
    # Check keyword match
    if keywords:
        if pd.notna(row['Overview']) and keywords.lower() in row['Overview'].lower():
            match_score += 1
    
    # Calculate percentage
    return (match_score / total_criteria) * 100

# Recommendation function with percentage calculation
def recommend_movies(genre, release_year, keywords):
    filtered_movies = movies_df
    
    if genre and genre != "Any":
        filtered_movies = filtered_movies[filtered_movies['Genres'].str.contains(genre, case=False, na=False)]
    if release_year and release_year != "Any":
        filtered_movies = filtered_movies[filtered_movies['Release_Year'] == int(release_year)]
    if keywords:
        filtered_movies = filtered_movies[filtered_movies['Overview'].str.contains(keywords, case=False, na=False)]
    
    # Calculate match percentages
    filtered_movies['match_percentage'] = filtered_movies.apply(lambda row: calculate_match_percentage(row, genre, release_year, keywords), axis=1)
    
    # Sort by match percentage and limit to top 5
    filtered_movies = filtered_movies.sort_values(by='match_percentage', ascending=False)
    top_movies = filtered_movies.head(5)[['Original_Title', 'match_percentage', 'poster_url']]
    
    # Prepare the result as a list of dictionaries
    result = []
    for _, row in top_movies.iterrows():
        result.append({
            'title': row['Original_Title'],
            'match_percentage': row['match_percentage'],
            'poster_url': row['poster_url']
        })
    
    return result

# Add a label and dropdown for genre selection
ttk.Label(root, text="Select a genre:").grid(column=0, row=0, padx=10, pady=10)
genre_input = ttk.Combobox(root, values=["Any"] + all_genres)
genre_input.grid(column=1, row=0, padx=10, pady=10)
genre_input.set("Any")

# Add a label and dropdown for release year selection
ttk.Label(root, text="Select a release year:").grid(column=0, row=1, padx=10, pady=10)
release_year_input = ttk.Combobox(root, values=["Any"] + [str(year) for year in all_release_years])
release_year_input.grid(column=1, row=1, padx=10, pady=10)
release_year_input.set("Any")

# Add a label and entry for keyword input
ttk.Label(root, text="Enter keywords:").grid(column=0, row=2, padx=10, pady=10)
keyword_input = ttk.Entry(root)
keyword_input.grid(column=1, row=2, padx=10, pady=10)

# Add a button to trigger recommendation
ttk.Button(root, text="Get Recommendations", command=show_recommendations).grid(column=0, row=3, columnspan=2, padx=10, pady=10)

# Create a frame to hold the results
result_frame = tk.Frame(root)
result_frame.grid(column=0, row=4, columnspan=2, padx=10, pady=10)

# Run the application
root.mainloop()



/var/folders/nq/ln_gwthx2kb0qfqrxc4zszmr0000gn/T/ipykernel_25075/4109838405.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_movies['match_percentage'] = filtered_movies.apply(lambda row: calculate_match_percentage(row, genre, release_year, keywords), axis=1)
Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/tkinter/__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/nq/ln_gwthx2kb0qfqrxc4zszmr0000gn/T/ipykernel_25075/4109838405.py", line 53, in show_recommendations
    img = Image.open(BytesIO(img_data))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/PIL/Image.py", line 3309

In [11]:
# Extract unique genres from the dataset
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [genre['name'] for genre in genres]
    except:
        return []

# Extract the release year from the release_date column
movies_df['Release_Year'] = pd.to_datetime(movies_df['Release_Date'], errors='coerce').dt.year

all_genres = set()
all_release_years = sorted(movies_df['Release_Year'].dropna().astype(int).unique(), reverse=True)

movies_df['Genres'].dropna().apply(lambda x: all_genres.update(extract_genres(x)))

all_genres = sorted(list(all_genres))

# Initialize the main window
root = tk.Tk()
root.title("Movie Recommender")

# Define the function to show recommendations
def show_recommendations():
    genre = genre_input.get()
    release_year = release_year_input.get()
    keywords = keyword_input.get()
    recommendations = recommend_movies(genre, release_year, keywords)
    result_var.set("\n".join(recommendations))

# Calculate match percentage based on genre, release year, and keywords
def calculate_match_percentage(row, genre, release_year, keywords):
    match_score = 0
    total_criteria = 3  # We have three criteria now: genre, release year, and keywords
    
    # Check genre match
    if genre and genre != "Any":
        if genre in extract_genres(row['Genres']):
            match_score += 1
    
    # Check release year match
    if release_year and release_year != "Any":
        if pd.notna(row['Release_Year']) and row['Release_Year'] == int(release_year):
            match_score += 1
            
    # Check keyword match
    if keywords:
        if pd.notna(row['Overview']) and keywords.lower() in row['Overview'].lower():
            match_score += 1
    
    # Calculate percentage
    return (match_score / total_criteria) * 100

# Recommendation function with percentage calculation
def recommend_movies(genre, release_year, keywords):
    filtered_movies = movies_df
    
    if genre and genre != "Any":
        filtered_movies = filtered_movies[filtered_movies['Genres'].str.contains(genre, case=False, na=False)]
    if release_year and release_year != "Any":
        filtered_movies = filtered_movies[filtered_movies['Release_Year'] == int(release_year)]
    if keywords:
        filtered_movies = filtered_movies[filtered_movies['Overview'].str.contains(keywords, case=False, na=False)]
    
    # Calculate match percentages
    filtered_movies['match_percentage'] = filtered_movies.apply(lambda row: calculate_match_percentage(row, genre, release_year, keywords), axis=1)
    
    # Sort by match percentage and limit to top 5
    filtered_movies = filtered_movies.sort_values(by='match_percentage', ascending=False)
    top_movies = filtered_movies.head(5)[['Original_Title', 'match_percentage','poster_url']]
    
    # Format the output
    result = [f"{row['Original_Title']} - {row['match_percentage']:.2f} {row['poster_url']}% " for _, row in top_movies.iterrows()]
    # print(result)
    return result

# Add a label and dropdown for genre selection
ttk.Label(root, text="Select a genre:").grid(column=0, row=0, padx=10, pady=10)
genre_input = ttk.Combobox(root, values=["Any"] + all_genres)
genre_input.grid(column=1, row=0, padx=10, pady=10)
genre_input.set("Any")

# Add a label and dropdown for release year selection
ttk.Label(root, text="Select a release year:").grid(column=0, row=1, padx=10, pady=10)
release_year_input = ttk.Combobox(root, values=["Any"] + [str(year) for year in all_release_years])
release_year_input.grid(column=1, row=1, padx=10, pady=10)
release_year_input.set("Any")

# Add a label and entry for keyword input
ttk.Label(root, text="Enter keywords:").grid(column=0, row=2, padx=10, pady=10)
keyword_input = ttk.Entry(root)
keyword_input.grid(column=1, row=2, padx=10, pady=10)

# Add a button to trigger recommendation
ttk.Button(root, text="Get Recommendations", command=show_recommendations).grid(column=0, row=3, columnspan=2, padx=10, pady=10)

# Add a label to display the results
result_var = tk.StringVar()
ttk.Label(root, textvariable=result_var).grid(column=0, row=4, columnspan=2, padx=10, pady=10)

# Run the application
root.mainloop()

/var/folders/nq/ln_gwthx2kb0qfqrxc4zszmr0000gn/T/ipykernel_25075/1728137445.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_movies['match_percentage'] = filtered_movies.apply(lambda row: calculate_match_percentage(row, genre, release_year, keywords), axis=1)


['Dangerous Minds - 33.33 https://image.tmdb.org/t/p/original/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg% ', 'Les Héritiers - 33.33 https://image.tmdb.org/t/p/original/74UW9qaDe7ahhMAQBziaMCVQKyM.jpg% ', 'Senior Project - 33.33 https://image.tmdb.org/t/p/original/sSEixqCVwgyjg1i2IGl2xl2mM5R.jpg% ', '말죽거리 잔혹사 - 33.33 https://image.tmdb.org/t/p/original/7UNbXips8TUZ1RTrRudRZ3FTRSh.jpg% ', 'Buster and Billie - 33.33 https://image.tmdb.org/t/p/original/47HR9Trqk83hpIgFg7JXIDeTDRh.jpg% ']


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

# Filter relevant columns and drop rows with missing values
data = movies_df[['vote_average', 'vote_count']].dropna()

# Feature Scaling
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# Apply Gaussian Mixture Model for clustering
gmm = GaussianMixture(n_components=3, random_state=42)  # Assuming 3 clusters, can be adjusted
gmm_labels = gmm.fit_predict(scaled_data)

# Add cluster labels to the original data
data['cluster'] = gmm_labels

# Plot the clusters
plt.figure(figsize=(10, 6))
plt.scatter(data['vote_average'], data['vote_count'], c=data['cluster'], cmap='viridis', s=50, alpha=0.7)
plt.title("Distribution-Based Clustering of Movies")
plt.xlabel("Vote Average")
plt.ylabel("Vote Count")
plt.colorbar(label='Cluster')
plt.show()


NameError: name 'movies_df' is not defined

In [12]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import ast

# Ensure 'Cast_parse' is in list format and handle any non-list or NaN entries
movies_df['Cast_parse'] = movies_df['Cast_parse'].apply(lambda x: eval(x) if isinstance(x, str) else x if isinstance(x, list) else [])

# Extract genres and ensure they are in the correct format
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [genre['name'] for genre in genres]
    except (ValueError, SyntaxError):
        return []

all_genres = set()
movies_df['Genres'].dropna().apply(lambda x: all_genres.update(extract_genres(x)))

all_genres = sorted(list(all_genres))

# Extract the release year from the release_date column
movies_df['Release_Year'] = pd.to_datetime(movies_df['Release_Date'], errors='coerce').dt.year
all_release_years = sorted(movies_df['Release_Year'].dropna().astype(int).unique(), reverse=True)

# Initialize the main window
root = tk.Tk()
root.title("Movie Recommender")

# Define the function to show recommendations based on actor
def show_actor_recommendations():
    actor_name = actor_input.get().strip().lower()
    filtered_movies = movies_df[movies_df['Cast_parse'].apply(lambda cast: any(actor_name == actor.lower().strip() for actor in cast))]
    
    if filtered_movies.empty:
        result_var.set(f"No movies found with actor '{actor_name.title()}'.")
    else:
        filtered_movies_sorted = filtered_movies.sort_values(by='Popularity', ascending=False)
        top_5_filtered_movies = filtered_movies_sorted.head(5)
        
        # Formatting the output
        results = [f"Title: {row['Original_Title']}\nVote Average: {row['Vote_Average']}\nPopularity: {row['Popularity']}\n{'-' * 50}" 
                   for _, row in top_5_filtered_movies.iterrows()]
        result_var.set("\n\n".join(results))

# Add a label and entry for actor name input
ttk.Label(root, text="Enter Actor's Name:").grid(column=0, row=0, padx=10, pady=10)
actor_input = ttk.Entry(root)
actor_input.grid(column=1, row=0, padx=10, pady=10)

# Add a button to trigger actor-based recommendation
ttk.Button(root, text="Find Movies", command=show_actor_recommendations).grid(column=0, row=1, columnspan=2, padx=10, pady=10)

# Add a label to display the results
result_var = tk.StringVar()
result_label = ttk.Label(root, textvariable=result_var, justify='left')
result_label.grid(column=0, row=2, columnspan=2, padx=10, pady=10)

# Run the application
root.mainloop()
